<a href="https://colab.research.google.com/github/melzismn/Digital-Design-2020-2021/blob/master/icp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ONLY FOR COLAB
# Not required in Binder

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local

!conda install -q -y --prefix /usr/local python=3.6 ujson

import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

import ujson
print(ujson.dumps({1:2}))

!conda install -c conda-forge igl
!conda install -c conda-forge meshplot

# Import necessary utils

In [ ]:
import igl
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact
from scipy.sparse.linalg import eigsh
from scipy.sparse import csr_matrix
import os 
from sklearn.cluster import KMeans
from pyflann import *

# Run a first example of flann search
In this exercise we will exploy the [FLANN Library](https://pypi.org/project/flann/) a Fast Library for Approximate Nearest Neighbors (FLANN)

In [ ]:
dataset = np.random.rand(10000, 128)
testset = np.random.rand(1000, 128)
flann = FLANN()
[result, dists] = flann.nn(dataset, testset, 1, algorithm="kmeans", branching=32, iterations=7, checks=16)
print result
print dists